Import nécéssaires

In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import re
import torch
import torchvision
import math
#Information
# epoch = one forward and backward pass of ALL training samples
# batch_size = number of training samples used in one forward/backward pass
# number of iterations = number of passes, each pass (forward+backward) using [batch_size] number of sampes
# e.g : 100 samples, batch_size=20 -> 100/20=5 iterations for 1 epoch

### **DATA**

Dans cette cellule :
**********
***lirefile*** vas lire le fichier en brut et renvoyé une list of list des proteine chaque proteine dans une list, et pour labels  également une list of list des targets.
*******
je définit les dictionnaires avec chaque acide et targets son code en one hote je le fait manuellment (ac_am_dict , targ_dict )

In [ ]:
def lirefile(file_name):
    data=[]
    target=[]
    
    
    with open(file_name, 'r', encoding='utf-8') as raw_file:
        for i,line in enumerate(raw_file):
            line_content = re.sub('[\n\t]', '', line)
            if line_content not in ["end",'<>']:
                d=line_content.split(" ")
                data[-1].append(d[0])
                target[-1].append(d[1])
            elif line_content in ['<>']:
                data.append([])
                target.append([])
    return data , target

In [ ]:
#one hot
ac_am_dict = {
    "1" :[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "A": [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "C": [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "D": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "E": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "F": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "G": [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "H": [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "I": [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "K": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "L": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "M": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "N": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    "P": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    "Q": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    "R": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    "S": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    "T": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    "V": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    "W": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    "Y": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    
}

targ_dict = {
    
    "_": [1,0,0],
    "e": [0,1,0],
    "h": [0,0,1],
    
}

Dans la cellule suivante je vais d'abord ajouté le padding si c'est la premiere ou derniere acide.

Une fois le padding ajouté à toute la proteine je construit la séquence de 5

In [ ]:
def transformAcideAminToSerieFive(data):
      cpt=0
      array=[]
      Finale = []
      for row in data:
          
        array=[]
        id=0
        cpt=cpt+(len(row))
        
        for id, val in enumerate(row):
                  if id==0   : #verifier first acide
                    array.append(ac_am_dict["1"])
                    array.append(ac_am_dict["1"]) 
                    array.append(ac_am_dict[val])
                    
                  elif id==len(row)-1 :#vérifier last acide
                    array.append(ac_am_dict[val])
                    array.append(ac_am_dict["1"])
                    array.append(ac_am_dict["1"]) 
                    
                  
                  else :
                  
                      array.append(ac_am_dict[val])
      
        for idx, val in enumerate(array):
              el=[]
              if (idx+1 < len(array) and idx - 1 >= 0 and idx+2 < len(array) and idx - 2 >= 0 ):#construction des séquence de 5 
                  avant2=idx - 2
                  avant1=idx - 1
                  cur=idx
                  apres1=idx+1
                  apres2=idx+2
                  
                  #el = np.concatenate((array[avant2],array[avant1],array[cur],array[apres2],array[apres1]))
                  el = [*array[avant2],*array[avant1],*array[cur],*array[apres2],*array[apres1]]#concaténation de des 5 élément 
                  print(el)
                  Finale.append(el)#et j'ajoute à la liste finale
      print('nombre d\'acide aminé',cpt)
      return Finale   

In [ ]:
#ici je transforme le target en one hot seulement pour l'acide du milieu
def TransformTarget(label):
    array2=[]
    FinaleTarget = []
    for row in label:
        
        array2=[]

        
        for va in row:

              FinaleTarget.append(targ_dict[va])
     
    return FinaleTarget

In [ ]:
#Je construit mon dataset qui vas etre l'entrée de mon dataloader sa sortie c'est de tensor
#C'est la derniere étape de Transformation de mes données.
class Dataset(Dataset):

    def __init__(self,FinaleX,FinaleY):
       

        # here the first column is the class label, the rest are the features
        #Alors ici je transform chaque séquence en tensor
        lst1 = [torch.from_numpy(np.array(item)).type(torch.float) for item in FinaleX]
        print(type(lst1))
        self.x_data = lst1
        #et ici je transform chaque target en tensor 
        lst2=[]
        for item in FinaleY :
            lst2.append(torch.from_numpy(np.array(item)).type(torch.float))
        
        self.y_data = lst2

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return len(self.y_data)





Train data 

111 111
nombre d'acide aminé 18105
18105
nombre de labels 18105
18105

In [ ]:
dataTrain , labelTrain =lirefile("/content/drive/MyDrive/AvignonM1_ApplicationIA/DATA/protein-secondary-structure (1).train")
print(len(dataTrain),len(labelTrain))


TrainFinaleX=transformAcideAminToSerieFive(dataTrain)    
#Print     
print(len(TrainFinaleX))
#np.array(TrainFinale[0:10])


TrainFinaleY=TransformTarget(labelTrain)
#print    
print('nombre de labels',len(TrainFinaleY))
print(len(TrainFinaleY))
#TrainFinaleY[0:150]




111 111
nombre d'acide aminé 18105
18105
nombre de labels 18105
18105


In [ ]:
#Create Dataset and call Dataloader
dataset = Dataset(TrainFinaleX,TrainFinaleY)
first_data = dataset[1]
features, labels = first_data
print(features, labels)
train_loader = DataLoader(dataset=dataset,batch_size=64, shuffle=True,num_workers=2)

<class 'list'>
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]) tensor([1., 0., 0.])


Test Data

17 17
nombre d'acide aminé 3520
3520
nombre de labels 3520
3520

In [ ]:
dataTest , labelTest =lirefile("/content/drive/MyDrive/AvignonM1_ApplicationIA/DATA/protein-secondary-structure.test")
print(len(dataTest),len(labelTest))

TestFinaleX=transformAcideAminToSerieFive(dataTest)    
#Print     
print(len(TestFinaleX))
#np.array(TrainFinaleX[0:10])


TestFinaleY=TransformTarget(labelTest)
#print    
print('nombre de labels',len(TestFinaleY))
print(len(TestFinaleY))
#TestFinaleY[0:150]




17 17
nombre d'acide aminé 3520
3520
nombre de labels 3520
3520


In [ ]:
#Create Dataset and call Dataloader

dataset = Dataset(TestFinaleX,TestFinaleY)
test_loader = DataLoader(dataset=dataset,batch_size=64, shuffle=True,num_workers=2)

<class 'list'>


# **Model**

In [ ]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

Model 1

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class NeuralNetwork(nn.Module):
    def __init__(self):

        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # je  Flattern pour faire  sortir un Tenseur 1D.
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(22*5, 256),
            nn.Linear(256, 128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 3)
        )
        self.drops = nn.Dropout(0.0)

    
        

    def forward(self, x):
        x = self.flatten(x)
        x=self.drops(x)
        logits = self.linear_relu_stack(x)
        out = F.softmax(logits)
        '''print(type(out))
        print(out.shape)'''
        return out

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=110, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=3, bias=True)
  )
  (drops): Dropout(p=0.0, inplace=False)
)


**Train and test loop**

In [ ]:
learning_rate = 0.1

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        
        pred = model(X)
        #print('z',z)
        print('y',y)
        print(pred)
        
        loss = loss_fn(pred, y)
        print(loss)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            #y=torch.flatten(y)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            print("testLoss",test_loss)
            z=pred.argmax(axis=1)

            print('z',z)
            print('y',y)
            print(pred)
            
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()#M'a causé longtemps une erreur,
          # très important pour comparé d'voir le target qui le maximum de prédiction  et on retourn 1 dans son index .EXMPL [0.65,0.20,0.15]=>[1,0,0]

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.88)


for t in range(10):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
    test_loop(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
y tensor([[0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1.,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.]